In [8]:
import pandas as pd
from functions import connectCollection, searchNear, searchNearWithoutLimit

df_filtered = pd.read_csv("../input/clean_df_companies.csv")
print("clean_df_companies.csv successfully imported")
# Import airports around
db, airports = connectCollection('companies', 'airports')
print("airports collection successfully loaded")

df_filtered["Closest_Airport"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, airports, 40000), axis=1)
df_filtered = df_filtered[df_filtered['Closest_Airport'].astype(bool)]

# Import old companies around
db, oldcos = connectCollection('companies', 'oldCos')
print("oldCos collection successfully loaded")

df_filtered["Closest_oldCo"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, oldcos, 300), axis=1)
df_filtered = df_filtered[df_filtered['Closest_oldCo'].astype(
    bool) == False]

# Import Starbucks around
db, starbucks = connectCollection('companies', 'starbucks')
print("starbucks collection successfully loaded")

df_filtered["Closest_Starbucks"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, starbucks, 1000), axis=1)
df_filtered = df_filtered[df_filtered['Closest_Starbucks'].astype(
    bool)]

# Import Tech Companies with >$1m raised around
db, techCos = connectCollection('companies', 'techCos')
print("techCos collection successfully loaded")

df_filtered["Closest_techCo"] = df_filtered.apply(lambda x: searchNearWithoutLimit(
    x.longitude, x.latitude, techCos, 400), axis=1)
df_filtered = df_filtered[df_filtered['Closest_techCo'].astype(
    bool)]
df_filtered.reset_index(inplace=True, drop=True)
df_filtered.to_csv("../input/df_filtered")
print("df_filtered.csv successfully exported")

# Checking which cities have more techCos around
df_filtered["Number_of_TechCo_around"] = df_filtered.Closest_techCo.apply(
    lambda x: len(x))
df_check_techCos = df_filtered[["city", "Number_of_TechCo_around"]]
df_check_techCos = df_check_techCos.groupby("city").sum().sort_values(
    by="Number_of_TechCo_around", ascending=False)

# Checking apartment rent prices
df_apartment = pd.read_csv("../input/apartment-rent-summary.csv")
df_apartment[df_apartment["Location"].str.contains(
    "Atlanta|Chicago|Denver|Austin|San Mateo") == True].sort_values(by=["Price_3br"], ascending=False)
'''
Austin will be chosen, as it has many tech companies around, good rental prices and outranked Silicon Valley as the top city for startups
http://austin.culturemap.com/news/innovation/07-03-19-austin-ranking-best-cities-startups-commercialcafe/
'''
df_austin = df_filtered[df_filtered.city == "Austin"]
df_austin.reset_index(inplace=True, drop=True)
df_austin.to_csv("../input/df_austin.csv")



clean_df_companies.csv successfully imported
airports collection successfully loaded
oldCos collection successfully loaded
starbucks collection successfully loaded
techCos collection successfully loaded
df_filtered.csv successfully exported


In [11]:
df_austin_final = df_austin.copy()

In [12]:
df_austin_trial.Closest_Airport[0][0]["geoJSON"]["coordinates"]

NameError: name 'df_austin_trial' is not defined

In [13]:
df_austin_trial.Closest_Starbucks[0][0]["geoJSON"]["coordinates"]

NameError: name 'df_austin_trial' is not defined

In [14]:
df_austin_final["Airport_Latitude"] = df_austin_final["Closest_Airport"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_austin_final["Airport_Longitude"] = df_austin_final["Closest_Airport"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])
df_austin_final["Starbucks_Latitude"] = df_austin_final["Closest_Starbucks"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_austin_final["Starbucks_Longitude"] = df_austin_final["Closest_Starbucks"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])
df_austin_final["TechCo_Latitude"] = df_austin_final["Closest_techCo"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_austin_final["TechCo_Longitude"] = df_austin_final["Closest_techCo"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])


In [15]:
df_austin_final

,Unnamed: 0,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,...,Closest_oldCo,Closest_Starbucks,Closest_techCo,Number_of_TechCo_around,Airport_Latitude,Airport_Longitude,Starbucks_Latitude,Starbucks_Longitude,TechCo_Latitude,TechCo_Longitude
0,890,52cdef7c4bab8bd67529809a,Shangby,shangby,http://www.crunchbase.com/company/shangby,http://www.shangby.com,http://www.shangby.com/site/news_letter,NaN,NaN,ecommerce,...,[],"[{'_id': 5dd67f43e1d2f64cdaf44024, 'city': 'Au...","[{'_id': '52cdef7c4bab8bd67529809a', 'Unnamed:...",2,30.202108,-97.666837,30.292404,-97.825104,30.288653,-97.822884
1,1490,52cdef7c4bab8bd6752982f7,CyberRentals,cyberrentals,http://www.crunchbase.com/company/cyberrentals,http://www.cyberrentals.com,NaN,NaN,NaN,web,...,[],"[{'_id': 5dd67f43e1d2f64cdaf42d9b, 'city': 'Au...","[{'_id': '52cdef7c4bab8bd6752982f7', 'Unnamed:...",2,30.202108,-97.666837,30.237675,-97.792549,30.242865,-97.798398
2,1492,52cdef7c4bab8bd6752982f9,TripHomes,triphomes,http://www.crunchbase.com/company/triphomes,http://www.HomeAway.com,NaN,NaN,NaN,NaN,...,[],"[{'_id': 5dd67f43e1d2f64cdaf42d9b, 'city': 'Au...","[{'_id': '52cdef7c4bab8bd6752982f7', 'Unnamed:...",2,30.202108,-97.666837,30.237675,-97.792549,30.242865,-97.798398
3,1688,52cdef7c4bab8bd6752983ba,MessageOne,messageone,http://www.crunchbase.com/company/messageone,NaN,NaN,NaN,NaN,NaN,...,[],"[{'_id': 5dd67f43e1d2f64cdaf42990, 'city': 'Au...","[{'_id': '52cdef7c4bab8bd6752983ba', 'Unnamed:...",1,30.202108,-97.666837,30.401922,-97.746902,30.405197,-97.745673
4,2571,52cdef7c4bab8bd67529873f,Absolute Software,absolute-software,http://www.crunchbase.com/company/absolute-sof...,http://www.absolute.com,http://blog.absolute.com,http://blog.absolute.com/feed/atom/,absolutecorp,software,...,[],"[{'_id': 5dd67f43e1d2f64cdaf44588, 'city': 'Au...","[{'_id': '52cdef7f4bab8bd67529c39e', 'Unnamed:...",2,30.202108,-97.666837,30.265875,-97.746445,30.264043,-97.744762
5,10563,52cdef7e4bab8bd67529ad8b,NetStreams,netstreams,http://www.crunchbase.com/company/netstreams,NaN,NaN,NaN,NaN,games_video,...,[],"[{'_id': 5dd67f43e1d2f64cdaf411a3, 'city': 'Au...","[{'_id': '52cdef7e4bab8bd67529ad8b', 'Unnamed:...",1,30.202108,-97.666837,30.418882,-97.702713,30.424305,-97.709981
6,12856,52cdef7e4bab8bd67529b8c0,Infochimps,infochimps,http://www.crunchbase.com/company/infochimps,http://infochimps.com,http://blog.infochimps.com/,http://blog.infochimps.com/feed,infochimps,enterprise,...,[],"[{'_id': 5dd67f43e1d2f64cdaf41de4, 'city': 'Au...","[{'_id': '52cdef7e4bab8bd67529b8c0', 'Unnamed:...",1,30.202108,-97.666837,30.269783,-97.753914,30.273482,-97.758883
7,13694,52cdef7e4bab8bd67529bcf8,AppUseful,appuseful,http://www.crunchbase.com/company/appuseful,http://appuseful.com,http://feeds2.feedburner.com/appuseful,http://appuseful.com/blog,usefulwebapps,web,...,[],"[{'_id': 5dd67f43e1d2f64cdaf41447, 'city': 'Au...","[{'_id': '52cdef7e4bab8bd67529bcf8', 'Unnamed:...",1,30.202108,-97.666837,30.302490,-97.738510,30.296171,-97.738954
8,14406,52cdef7f4bab8bd67529c01d,Quintiles,quintiles,http://www.crunchbase.com/company/quintiles,http://www.quintiles.com,NaN,NaN,NaN,biotech,...,[],"[{'_id': 5dd67f43e1d2f64cdaf4302f, 'city': 'Au...","[{'_id': '52cdef7f4bab8bd67529c01d', 'Unnamed:...",2,30.202108,-97.666837,30.220448,-97.757416,30.213039,-97.751754
9,15124,52cdef7f4bab8bd67529c39e,Pelorus Technology,pelorus-technology,http://www.crunchbase.com/company/pelorus-tech...,http://pelorustechnology.com,NaN,NaN,NaN,enterprise,...,[],"[{'_id': 5dd67f43e1d2f64cdaf44e60, 'city': 'Au...","[{'_id': '52cdef7f4bab8bd67529c39e', 'Unnamed:...",8,30.202108,-97.666837,30.264475,-97.743774,30.264043,-97.744762


In [16]:
import folium
from folium.plugins import HeatMap

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')  

# Create map with release incidents and monitoring stations
m = folium.Map(location=[30.288653 ,-97.822884], zoom_start=11)
HeatMap(data=df_austin_final[['latitude', 'longitude']], radius=50).add_to(m)
for idx, row in df_austin_final.iterrows():
    folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(icon='home',color='blue')).add_to(m)
    folium.Marker([row['Starbucks_Latitude'], row['Starbucks_Longitude']], icon=folium.Icon(icon='cutlery',color='darkgreen')).add_to(m)
#     folium.Marker([row['TechCo_Latitude'], row['TechCo_Longitude']], icon=folium.Icon(icon='briefcase',color='black')).add_to(m)
    folium.Marker([row['Airport_Latitude'], row['Airport_Longitude']], icon=folium.Icon(icon='plane',color='red')).add_to(m)
           
# Plot each polygon on the map
# GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
embed_map(m, 'm1.html')

In [17]:
df_filtered

,Unnamed: 0,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,...,state_code,country_code,latitude,longitude,Tech/Other,Closest_Airport,Closest_oldCo,Closest_Starbucks,Closest_techCo,Number_of_TechCo_around
0,11,52cdef7c4bab8bd675297d99,Lala,lala,http://www.crunchbase.com/company/lala,http://www.lala.com,http://www.lala.com/#blog,http://www.lala.com/#blog,NaN,games_video,...,CA,USA,37.451151,-122.154369,Tech,"[{'_id': 5dd675f33b0eaf5b22d77dad, 'name': 'Sa...",[],"[{'_id': 5dd67f43e1d2f64cdaf41b93, 'city': 'Pa...","[{'_id': '52cdef7c4bab8bd675297d99', 'Unnamed:...",1
1,107,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,MI,USA,42.547807,-83.215498,Other,"[{'_id': 5dd675f33b0eaf5b22d77c91, 'name': 'De...",[],"[{'_id': 5dd67f43e1d2f64cdaf423e5, 'city': 'Bi...","[{'_id': '52cdef7c4bab8bd675297e67', 'Unnamed:...",1
2,108,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,CA,USA,33.658331,-117.861817,Other,"[{'_id': 5dd675f33b0eaf5b22d77dae, 'name': 'Jo...",[],"[{'_id': 5dd67f43e1d2f64cdaf4315c, 'city': 'Ne...","[{'_id': '52cdef7e4bab8bd67529bd27', 'Unnamed:...",1
3,117,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,DC,USA,38.901113,-77.027101,Other,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf41177, 'city': 'Wa...","[{'_id': '52cdef7d4bab8bd6752995f9', 'Unnamed:...",3
4,267,52cdef7c4bab8bd675297e47,ScanScout,scanscout,http://www.crunchbase.com/company/scanscout,http://www.scanscout.com,NaN,NaN,NaN,advertising,...,CA,USA,34.050887,-118.459995,Other,"[{'_id': 5dd675f33b0eaf5b22d77eb9, 'name': 'Lo...",[],"[{'_id': 5dd67f43e1d2f64cdaf428e3, 'city': 'Lo...","[{'_id': '52cdef7e4bab8bd67529a28d', 'Unnamed:...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,15632,52cdef7f4bab8bd67529c5e8,Shape Games,shape-games,http://www.crunchbase.com/company/shape-games,http://www.shapegame.ru,NaN,NaN,NaN,games_video,...,VA,USA,38.814208,-77.042423,Tech,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf4287c, 'city': 'Al...","[{'_id': '52cdef7f4bab8bd67529c5e8', 'Unnamed:...",2
457,15668,52cdef7f4bab8bd67529c60a,Technologies and Devices International,technologies-and-devices-international,http://www.crunchbase.com/company/technologies...,http://www.tdii.com,NaN,NaN,NaN,semiconductor,...,MD,USA,39.053372,-76.958117,Tech,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf4265a, 'city': 'Si...","[{'_id': '52cdef7f4bab8bd67529c60a', 'Unnamed:...",1
458,15677,52cdef7f4bab8bd67529c613,KooDooZ,koodooz,http://www.crunchbase.com/company/koodooz,http://www.koodooz.com,http://koodooz.blogspot.com,http://www.addthis.com/feed.php?pub=koodooz&h1...,KooDooZ,other,...,CA,USA,34.013266,-118.490269,Other,"[{'_id': 5dd675f33b0eaf5b22d77eb9, 'name': 'Lo...",[],"[{'_id': 5dd67f43e1d2f64cdaf45697, 'city': 'Sa...","[{'_id': '52cdef7f4bab8bd67529c613', 'Unnamed:...",4
459,15834,52cdef7f4bab8bd67529c6e4,Getyoo,getyoo,http://www.crunchbase.com/company/getyoo,http://www.getyoo.com,http://getyoo.tumblr.com/,http://getyoo.tumblr.com/rss,getyoo,mobile,...,NaN,BEL,50.839970,4.346472,Tech,"[{'_id': 5dd675f33b0eaf5b22d77d92, 'name': 'Br...",[],"[{'_id': 5dd67f43e1d2f64cdaf45ce5, 'city': 'Br...","[{'_id': '52cdef7f4bab8bd67529c6e4', 'Unnamed:...",1


In [44]:


map_city=folium.Map(location=[30.288653 ,-97.822884], zoom_start=12)
HeatMap(data=df_filtered[['latitude', 'longitude']], radius=50).add_to(map_city)
for starbucks in df_filtered.Closest_Starbucks:
    folium.Marker(starbucks[0]["geoJSON"]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
for tech in df_filtered.Closest_techCo:
    folium.Marker(tech[0]["geoJSON"]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='headphones',color='orange'), 
                   ).add_to(map_city)
for airport in df_filtered.Closest_Airport:
    folium.Marker(airport[0]["geoJSON"]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='plane',color='darkgreen'), 
                   ).add_to(map_city)
for idx, row in df_filtered.iterrows():
    folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(icon='home',color='blue')).add_to(map_city)
    
embed_map(map_city, 'm1.html')


In [24]:
df_filtered.Closest_Starbucks[0][0]["geoJSON"]

{'type': 'Point', 'coordinates': [-122.1605300903, 37.4467353821]}

In [45]:
df_filtered["Airport_Latitude"] = df_filtered["Closest_Airport"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_filtered["Airport_Longitude"] = df_filtered["Closest_Airport"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])
df_filtered["Starbucks_Latitude"] = df_filtered["Closest_Starbucks"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_filtered["Starbucks_Longitude"] = df_filtered["Closest_Starbucks"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])
df_filtered["TechCo_Latitude"] = df_filtered["Closest_techCo"].apply(lambda x: x[0]["geoJSON"]["coordinates"][1])
df_filtered["TechCo_Longitude"] = df_filtered["Closest_techCo"].apply(lambda x: x[0]["geoJSON"]["coordinates"][0])


In [46]:
df_filtered

,Unnamed: 0,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,...,Closest_oldCo,Closest_Starbucks,Closest_techCo,Number_of_TechCo_around,Airport_Latitude,Airport_Longitude,Starbucks_Latitude,Starbucks_Longitude,TechCo_Latitude,TechCo_Longitude
0,11,52cdef7c4bab8bd675297d99,Lala,lala,http://www.crunchbase.com/company/lala,http://www.lala.com,http://www.lala.com/#blog,http://www.lala.com/#blog,NaN,games_video,...,[],"[{'_id': 5dd67f43e1d2f64cdaf41b93, 'city': 'Pa...","[{'_id': '52cdef7c4bab8bd675297d99', 'Unnamed:...",1,37.369491,-121.929429,37.446735,-122.160530,37.451151,-122.154369
1,107,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,[],"[{'_id': 5dd67f43e1d2f64cdaf423e5, 'city': 'Bi...","[{'_id': '52cdef7c4bab8bd675297e67', 'Unnamed:...",1,42.409094,-83.003968,42.546398,-83.213730,42.545787,-83.215454
2,108,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,[],"[{'_id': 5dd67f43e1d2f64cdaf4315c, 'city': 'Ne...","[{'_id': '52cdef7e4bab8bd67529bd27', 'Unnamed:...",1,33.679486,-117.861489,33.656796,-117.865570,33.658546,-117.865950
3,117,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,[],"[{'_id': 5dd67f43e1d2f64cdaf41177, 'city': 'Wa...","[{'_id': '52cdef7d4bab8bd6752995f9', 'Unnamed:...",3,38.853716,-77.043337,38.901817,-77.025597,38.897976,-77.028561
4,267,52cdef7c4bab8bd675297e47,ScanScout,scanscout,http://www.crunchbase.com/company/scanscout,http://www.scanscout.com,NaN,NaN,NaN,advertising,...,[],"[{'_id': 5dd67f43e1d2f64cdaf428e3, 'city': 'Lo...","[{'_id': '52cdef7e4bab8bd67529a28d', 'Unnamed:...",1,33.944174,-118.402469,34.053726,-118.466125,34.048524,-118.462363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,15632,52cdef7f4bab8bd67529c5e8,Shape Games,shape-games,http://www.crunchbase.com/company/shape-games,http://www.shapegame.ru,NaN,NaN,NaN,games_video,...,[],"[{'_id': 5dd67f43e1d2f64cdaf4287c, 'city': 'Al...","[{'_id': '52cdef7f4bab8bd67529c5e8', 'Unnamed:...",2,38.853716,-77.043337,38.811478,-77.045273,38.814208,-77.042423
457,15668,52cdef7f4bab8bd67529c60a,Technologies and Devices International,technologies-and-devices-international,http://www.crunchbase.com/company/technologies...,http://www.tdii.com,NaN,NaN,NaN,semiconductor,...,[],"[{'_id': 5dd67f43e1d2f64cdaf4265a, 'city': 'Si...","[{'_id': '52cdef7f4bab8bd67529c60a', 'Unnamed:...",1,38.853716,-77.043337,39.055305,-76.954811,39.053371,-76.958117
458,15677,52cdef7f4bab8bd67529c613,KooDooZ,koodooz,http://www.crunchbase.com/company/koodooz,http://www.koodooz.com,http://koodooz.blogspot.com,http://www.addthis.com/feed.php?pub=koodooz&h1...,KooDooZ,other,...,[],"[{'_id': 5dd67f43e1d2f64cdaf45697, 'city': 'Sa...","[{'_id': '52cdef7f4bab8bd67529c613', 'Unnamed:...",4,33.944174,-118.402469,34.013973,-118.493195,34.013266,-118.490269
459,15834,52cdef7f4bab8bd67529c6e4,Getyoo,getyoo,http://www.crunchbase.com/company/getyoo,http://www.getyoo.com,http://getyoo.tumblr.com/,http://getyoo.tumblr.com/rss,getyoo,mobile,...,[],"[{'_id': 5dd67f43e1d2f64cdaf45ce5, 'city': 'Br...","[{'_id': '52cdef7f4bab8bd67529c6e4', 'Unnamed:...",1,50.897295,4.484640,50.834332,4.352340,50.839970,4.346472


In [65]:
import folium
from folium.plugins import HeatMap, MarkerCluster
import math

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')  

m = folium.Map(location=[40.4893538  ,-3.6827461 ], zoom_start=2)
# HeatMap(data=df_filtered[['TechCo_Latitude', 'TechCo_Longitude']], radius=50).add_to(m)
# for idx, row in df_filtered.iterrows():
#     folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(icon='home',color='blue')).add_to(m)
#     folium.Marker([row['Starbucks_Latitude'], row['Starbucks_Longitude']], icon=folium.Icon(icon='cutlery',color='darkgreen')).add_to(m)
# #     folium.Marker([row['TechCo_Latitude'], row['TechCo_Longitude']], icon=folium.Icon(icon='briefcase',color='black')).add_to(m)
#     folium.Marker([row['Airport_Latitude'], row['Airport_Longitude']], icon=folium.Icon(icon='plane',color='red')).add_to(m)

mc = MarkerCluster()
for idx, row in df_filtered.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(folium.Marker([row['latitude'], row['longitude']]))
m.add_child(mc)
HeatMap(data=df_filtered[['TechCo_Latitude', 'TechCo_Longitude']], radius=50).add_to(mc)

    
# Plot each polygon on the map
# GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
embed_map(m, 'm1.html')